# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
raw_df = pd.read_csv('../output_data/WeatherPy_output/raw_weather.csv')
print(len(raw_df))
raw_df.head()

619


,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
0,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-10800.0,Belmonte,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,-36000.0,Hilo,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,3600.0,Honningsvåg,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-3600.0,Lagoa,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,801.0,Clouds,few clouds,02n,stations,9000.0,1.608738e+09,28800.0,George Town,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
import gmaps
gmaps.configure(api_key=g_key)

In [35]:
coords = raw_df[['coord.lat', 'coord.lon']].fillna(0)
humidity = raw_df['main.humidity']

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [91]:
ideal_temp_mask = (raw_df['main.temp_max'] < 80) & (raw_df['main.temp_max'] > 70)
ideal_wind_mask = raw_df['wind.speed'] < 10
ideal_cloud_mask = raw_df['clouds.all'] == 0 

ideal_spots_df = raw_df[ideal_temp_mask & ideal_wind_mask & ideal_cloud_mask]
print(len(raw_df))
ideal_spots_df.head()

22


,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
11,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Tessalit,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,7200.0,Bredasdorp,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,-10800.0,São Pedro do Sul,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,34200.0,Yulara,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Taoudenni,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [92]:
hotel_coords= ideal_spots_df[['coord.lat','coord.lon']]
string_coords = []
responses = []

for vacation_spot in range(len(hotel_coords)):
    str_coord = f'{str(hotel_coords.iloc[vacation_spot, 0])}, {str(hotel_coords.iloc[vacation_spot,1])}'
    string_coords.append(str_coord)

target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

for vacation_spot in string_coords: 
    coords = vacation_spot
    
    # set up a parameters dictionary
    params = {
        "location": coords,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    responses.append(response)

In [93]:
clean_responses = [resp for resp in responses if resp.json()['status'] != 'ZERO_RESULTS']

In [100]:
hotel_df = {'hotel_name': []}

for response in clean_responses:
    hotel_json = response.json()
    hotel_df['hotel_name'].append(hotel_json['results'][0]['name'])

hotel_df = pd.DataFrame(hotel_df)
ideal_spots_df['Hotel Name'] = hotel_df
ideal_spots_df
# from pprint import pprint
# hotel_name = clean_responses[7].json()['results'][0]['name']
# hotel_coords = clean_responses[7].json()['results'][0]['geometry']['location']

,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h,hotel_name,Hotel Name
11,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Tessalit,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juddi Hotel,Juddi Hotel
74,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,7200.0,Bredasdorp,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,-10800.0,São Pedro do Sul,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,34200.0,Yulara,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Taoudenni,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,10800.0,Najrān,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,28800.0,Carnarvon,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Adrar,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,761.0,Dust,dust,50d,stations,2000.0,1.608738e+09,0.0,Nouadhibou,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,-10800.0,Cidreira,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
